In [57]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/sentiment-roberta-large-english-3-classes", return_all_scores=True)

#new prompt
#run it through the graph 
#maybe upload textattack tutorial? maybe it's still useful or will be in the future...

Some weights of the model checkpoint at j-hartmann/sentiment-roberta-large-english-3-classes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\14158\miniconda3\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False

In [56]:
def RunSentiment(Input_String:str) -> list:
    
    """
    Returns the Sentiment of Input String
    Output Will Return a list of negative sentiment and positive sentiment
    Example: [0.2, 0.8]
    """
    outputs = classifier(Input_String)
    negative_score = outputs[0][0]["score"]
    neutral_score = outputs[0][1]["score"]
    positive_score = outputs[0][2]["score"]
    total = negative_score + neutral_score + positive_score
    positive_score = ( positive_score + (neutral_score/2) ) / total
    negative_score = ( negative_score + (neutral_score/2) ) / total
    # print("negative score:",negative_score, "positive score:",positive_score)
    return [negative_score, positive_score]


In [94]:
%reload_ext autoreload
%autoreload 2
import textattack
from textattack.transformations import WordSwapEmbedding
from textattack.transformations import CompositeTransformation

from textattack.constraints.grammaticality import LanguageTool
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.pre_transformation import RepeatModification

from textattack.augmentation import Augmenter
# Set up transformation using CompositeTransformation()

def Augment(Input_String:str)-> list:
    transformation = CompositeTransformation(
        [WordSwapEmbedding(max_candidates=200)]
    )
    # WordSwapQWERTY()
    # Set up constraints


    words_list = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
    'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him',
    'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its',
    'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what',
    'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am',
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has',
    'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
    'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
    'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
    'through', 'during', 'before', 'after', 'above', 'below', 'to',
    'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
    'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
    'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same',
    'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don',
    'should', 'now'
]

    constraints = [StopwordModification(stopwords=words_list)]
    # Create augmenter with specified parameters
    augmenter = Augmenter(
        transformation=transformation,
        constraints=constraints,
        pct_words_to_swap=0.4,
        transformations_per_example=200,
    )
    # Augment!
    augment_list = augmenter.augment(Input_String)
    print(len(augment_list))
    print(augment_list)
    return augment_list

In [72]:
Augment("this is a positive sentence")



TRANSFORMATIONS # 200
89


['this is a active sentence',
 'this is a actively sentence',
 'this is a advantageous sentence',
 'this is a affirmative condemnation',
 'this is a affirmative convictions',
 'this is a affirmative penalties',
 'this is a affirmative penalty',
 'this is a affirmative sanctions',
 'this is a affirmative sentence',
 'this is a affirmative sentences',
 'this is a aggressively sentence',
 'this is a auspicious condemnation',
 'this is a auspicious convictions',
 'this is a auspicious pena',
 'this is a auspicious penalties',
 'this is a auspicious penalty',
 'this is a auspicious punishment',
 'this is a auspicious punishments',
 'this is a auspicious sanctions',
 'this is a auspicious sentence',
 'this is a auspicious sentences',
 'this is a auspicious sentencing',
 'this is a beneficial condemnation',
 'this is a beneficial penalties',
 'this is a beneficial sanctions',
 'this is a beneficial sentence',
 'this is a conducive pena',
 'this is a conducive sanctions',
 'this is a conducive

In [95]:
#GOAL: FROM Positive -> Negative

INPUT_STRING = "I'm feeling great today"

def simulate(INPUT_STRING:str, Radius:float, iterations:int): 

    Linear_Path = []
    Linear_Path.append(INPUT_STRING)
    Largest_Val = 0
    for _ in range(0,iterations):
            
        #Returns List of [Negative_Score, Positive_Score]
        INITIAL_Sentiment_Output = RunSentiment(INPUT_STRING)

        #Returns a List of Augmentations
        Augment_Output = Augment(INPUT_STRING)

        
        for item in Augment_Output:
            Negative_Output = RunSentiment(item)[0] 
            if INITIAL_Sentiment_Output[0] + Radius >= Negative_Output and Negative_Output >INITIAL_Sentiment_Output[0]:
            # if INITIAL_Sentiment_Output[0] < Negative_Output:

                if Negative_Output > Largest_Val:
                    Largest_Val = Negative_Output
                    INPUT_STRING = item
        if Largest_Val == INITIAL_Sentiment_Output[0]:
            print("loop ended early due to finding no gain in negativity")
            return Linear_Path
        print("Input string for next iteration is: ",INPUT_STRING)
        Linear_Path.append(INPUT_STRING)
    return Linear_Path

        

FINAL_ANSWER = simulate("this is positive", 0.175, 20)
print("FINAL LIST:", FINAL_ANSWER)


101
['this is actionable', 'this is active', 'this is actively', 'this is advantageous', 'this is advisable', 'this is affirmative', 'this is aggressively', 'this is agreeable', 'this is auspicious', 'this is beneficial', 'this is benevolent', 'this is charitable', 'this is comforting', 'this is conducive', 'this is congenial', 'this is constructive', 'this is corrective', 'this is decent', 'this is dynamic', 'this is dynamics', 'this is effective', 'this is efficacious', 'this is enabling', 'this is encouraging', 'this is energetic', 'this is enthusiastically', 'this is favor', 'this is favorable', 'this is favorably', 'this is favore', 'this is favored', 'this is favour', 'this is favourable', 'this is favourably', 'this is favoured', 'this is favouring', 'this is favours', 'this is fecund', 'this is fertile', 'this is fortunate', 'this is foster', 'this is fostered', 'this is fostering', 'this is fruitful', 'this is good', 'this is gratifying', 'this is handy', 'this is healthful', 

FINAL LIST: ['This is a positive sentence', 'This is a favorable convictions', 'Comme is a favorable convictions', 'Même is a favorable sentencing', 'Autre is a positive sentencing', 'Ça is a positive sentencing', 'Peux is a favorable sentencing', 'Peux is a supportive punishments', 'Vais is a supportive penalties', 'Vais is a conducive sentences', 'Vais is a propitious sentence', 'Vais is a propitious sentence', 'Peut is a propitious pena', 'Peux is a positive pena', 'Veux is a positive pina', 'Veux is a positive sentences', 'Veut is a beneficial sentences', 'Où is a beneficial punishments', 'Peut is a beneficial punishment', 'Peut is a conducive punish', 'Peut is a conducive punish']



In [93]:
#FINAL LIST: ['this is positive', 'this is enabling', 'this is favourable', 'this is sympathetic', 'this is lenient', 'this is resistant', 'this is endure', 'this is usury', 'this is greed', 'this is dishonesty', 'this is untruthful']
op = ['this is positive', 'this is enabling', 'this is favourable', 'this is sympathetic', 'this is lenient', 'this is resistant', 'this is endure', 'this is usury', 'this is greed', 'this is dishonesty', 'this is untruthful']
for item in op:
    print(RunSentiment(item))

[0.003929078962277364, 0.9960709210377227]
[0.16706280041816488, 0.8329371995818351]
[0.2949080219211754, 0.7050919780788246]
[0.46665751586275084, 0.5333424841372492]
[0.6392551929313546, 0.3607448070686454]
[0.791743086477946, 0.20825691352205403]
[0.8695426854581385, 0.13045731454186146]
[0.9994319592605706, 0.0005680407394293454]
[0.9994545146975465, 0.0005454853024534568]
[0.9994711916735233, 0.000528808326476781]
[0.9995120885529557, 0.0004879114470442703]
